In [ ]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

# Add project root to path
sys.path.append(os.path.abspath(os.path.join('..')))

from training.fractal_clustering import PatternTemplate # Needed for unpickling

plt.style.use('dark_background')
pd.set_option('display.max_columns', 500)
print("Dashboard initialized.")

In [ ]:
# Paths
LIBRARY_PATH = '../models/pattern_library.pkl' # Adjust if in checkpoints/
MC_STATE_PATH = '../checkpoints/mc_sweep_state.pkl'

# Load Pattern Library
try:
    with open(LIBRARY_PATH, 'rb') as f:
        library = pickle.load(f)
    print(f"✅ Loaded Pattern Library: {len(library)} active templates.")
except FileNotFoundError:
    print("❌ Pattern Library not found. Run Orchestrator first.")
    library = {}

# Load Monte Carlo Results
try:
    with open(MC_STATE_PATH, 'rb') as f:
        mc_data = pickle.load(f)
    results_db = mc_data.get('results_db', [])
    print(f"✅ Loaded MC Results: {len(results_db)} simulations.")
except FileNotFoundError:
    print("⚠️ No Monte Carlo sweep data found.")
    results_db = []

In [ ]:
# Extract Template Metadata
template_data = []
for t_id, data in library.items():
    # Handle different data structures (Dict vs Object)
    if isinstance(data, dict):
        # Legacy/Dict format
        z = data.get('centroid', [0,0,0,0])[0]
        mom = data.get('centroid', [0,0,0,0])[2]
        count = data.get('count', 0)
        pnl = data.get('params', {}).get('last_pnl', 0)
    else:
        # Object format
        z = data.centroid[0]
        mom = data.centroid[2]
        count = data.member_count
        pnl = 0 # Need to fetch from validation log if available

    template_data.append({
        'id': t_id,
        'z_score': abs(z),
        'momentum': abs(mom),
        'count': count,
        'pnl': pnl
    })

df_templates = pd.DataFrame(template_data)

if not df_templates.empty:
    plt.figure(figsize=(12, 8))
    scatter = plt.scatter(
        df_templates['z_score'], 
        df_templates['momentum'], 
        c=df_templates['count'], 
        cmap='viridis', 
        s=df_templates['count']*2, 
        alpha=0.7, 
        edgecolors='w'
    )
    plt.colorbar(scatter, label='Observation Count')
    plt.title('FRACTAL PHYSICS MANIFOLD\n(Where do our patterns live?)', fontsize=16)
    plt.xlabel('Mean Reversion Pressure (Z-Score)', fontsize=12)
    plt.ylabel('Trend Velocity (Momentum)', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.3)
    
    # Label top clusters
    for i, row in df_templates.nlargest(10, 'count').iterrows():
        plt.annotate(f"T_{row['id']}", (row['z_score'], row['momentum']), color='yellow')
        
    plt.show()
else:
    print("No template data to visualize.")

In [ ]:
if results_db:
    df_mc = pd.DataFrame(results_db)
    
    # Group by Template ID to see best performers
    best_strategies = df_mc.sort_values('sharpe_ratio', ascending=False).head(15)
    
    print("🏆 TOP 15 PERFORMING FRACTAL STRATEGIES")
    display(best_strategies[['template_id', 'timeframe', 'sharpe_ratio', 'total_pnl', 'win_rate', 'trades_count']])
else:
    print("No Monte Carlo data. Showing raw template stats instead:")
    if not df_templates.empty:
        display(df_templates.sort_values('count', ascending=False).head(10))